In [ ]:
from urllib.request import urlopen
import json
import pandas as pd
import plotly.graph_objects as go

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})

In [ ]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})
fig = go.Figure(go.Choroplethmapbox(geojson=counties, locations=df.fips, z=df.unemp,
                                    colorscale="Viridis", zmin=0, zmax=12,
                                    marker_opacity=0.5, marker_line_width=0))
fig.update_layout(mapbox_style="open-street-map",
                  mapbox_zoom=3, mapbox_center = {"lat": 37.0902, "lon": -95.7129})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
df.head()

In [ ]:
from urllib.request import urlopen
import json
import pandas as pd
import plotly.graph_objects as go

# Load county geojson data
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

# Load unemployment data
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv", dtype={"fips": str})

# Create choropleth map
fig = go.Figure(go.Choroplethmapbox(geojson=counties,
                                    locations=df.fips,
                                    z=df.unemp,
                                    colorscale="Viridis",
                                    zmin=0,
                                    zmax=12,
                                    marker_opacity=0.5,
                                    marker_line_width=0))

# Add scatter plot with longitude and latitude within the same mapbox layer
fig.add_trace(go.Scattermapbox(
    lon=[-100, -90, -80, -70],  # Example longitude data
    lat=[35, 40, 45, 50],       # Example latitude data
    mode='markers',
    marker=dict(
        size=10,
        color='red',
        opacity=0.8,
    ),
    text=['Marker 1', 'Marker 2', 'Marker 3', 'Marker 4'],  # Example text labels
))

# Update layout for mapbox
fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_zoom=3,
    mapbox_center={"lat": 37.0902, "lon": -95.7129}
)

# Update overall layout
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

# Show the figure
fig.show()


In [ ]:
import pydeck as pdk

UK_ACCIDENTS_DATA = 'https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/3d-heatmap/heatmap-data.csv'

layer = pdk.Layer(
    'HexagonLayer',  # `type` positional argument is here
    UK_ACCIDENTS_DATA,
    get_position=['lng', 'lat'],
    auto_highlight=True,
    elevation_scale=50,
    pickable=True,
    elevation_range=[0, 3000],
    extruded=True,
    coverage=1)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=-1.415,
    latitude=52.2323,
    zoom=6,
    min_zoom=5,
    max_zoom=15,
    pitch=40.5,
    bearing=-27.36
)

# Combined all of it and render a viewport
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html('hexagon-example.html')

In [ ]:
import pydeck

DATA_URL = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/geojson/vancouver-blocks.json"
LAND_COVER = [[[-123.0, 49.196], [-123.0, 49.324], [-123.306, 49.324], [-123.306, 49.196]]]

INITIAL_VIEW_STATE = pydeck.ViewState(
  latitude=49.254,
  longitude=-123.13,
  zoom=11,
  max_zoom=16,
  pitch=45,
  bearing=-27.36
)

polygon = pydeck.Layer(
    'PolygonLayer',
    LAND_COVER,
    stroked=False,
    # processes the data as a flat longitude-latitude pair
    get_polygon='-',
    get_fill_color=[0, 0, 0, 20]
)

geojson = pydeck.Layer(
    'GeoJsonLayer',
    DATA_URL,
    opacity=0.8,
    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    get_elevation='properties.valuePerSqm / 20',
    get_fill_color='[255, 255, properties.growth * 255]',
    get_line_color=[255, 255, 255],
    auto_highlight=True,

    pickable=True
)

r = pydeck.Deck(
    layers=[polygon, geojson],
    initial_view_state=INITIAL_VIEW_STATE)

r.to_html()

In [ ]:
import json
import pandas as pd
import pydeck as pdk
from urllib.request import urlopen

# Load county GeoJSON data
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

# Load unemployment data
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv", dtype={"fips": str})

# Update properties in GeoJSON data
for i in range(len(counties['features'])):
    feature = counties['features'][i]
    fips = feature['properties']['GEO_ID'][9:]  # Extract FIPS code from GEO_ID
    if fips in df['fips'].values:
        unemp_value = df[df['fips'] == fips]['unemp'].values[0]  # Get 'unemp' value for the FIPS code
        feature['properties']['unemp'] = unemp_value  # Adjust scaling as needed
        feature['properties']['custom_category'] = unemp_value  # Update custom_category
    else:
        feature['properties']['unemp'] = None  # or any default value if 'fips' not in df
        feature['properties']['custom_category'] = None  # or any default value for custom_category
    counties['features'][i] = feature

# Normalize 'unemp' values between 0 and 1, handling None values
max_unemp = df['unemp'].max()
min_unemp = df['unemp'].min()
counties['features'] = [
    {
        **feature,
        'properties': {
            **feature['properties'],
            'unemp': ((feature['properties']['unemp'] - min_unemp) / (max_unemp - min_unemp))*500000
            if feature['properties']['unemp'] is not None else None
        }
    }
    for feature in counties['features']
]

# Set the viewport location
view_state = pdk.ViewState(
    latitude=37.0902,
    longitude=-95.7129,
    zoom=3,
    min_zoom=0,
    max_zoom=15,
    pitch=0,
    bearing=0,
)

# Create a PolygonLayer for land cover
polygon_layer = pdk.Layer(
    'PolygonLayer',
    LAND_COVER,
    stroked=False,
    get_polygon='-',
    get_fill_color=[0, 0, 0, 20],
)


TOOLTIP_HTML ={
   "html": 
    '<b>Unemployement in U.S. counties</b><br />'+
    '<b>County: </b>: {properties.NAME}<br />'+
    "<b>Unemployment Value:</b> {properties.custom_category}",
   "style": {
        "backgroundColor": "olivedrab",
        "color": "white"
   }
}
# Create a GeoJsonLayer for the choropleth map
geojson_layer = pdk.Layer(
    'GeoJsonLayer',
    counties,
    opacity=0.3,
    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    get_elevation='properties.unemp',
    get_fill_color='[102,properties.custom_category*21,0]',
    get_line_color=[0, 0, 0],
    auto_highlight=True,
    pickable=True,
    #tooltip={'html': TOOLTIP_HTML} # Correct property names in the tooltip
)

# Combine all of it and render a viewport
deck = pdk.Deck(layers=[geojson_layer], initial_view_state=view_state, tooltip = TOOLTIP_HTML)
deck.to_html("pydeck_choropleth_map.html")

In [ ]:
counties['features'][0]


In [ ]:
TOOLTIP_HTML

In [ ]:
counties

In [ ]:
# Load county geojson data
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

# Load unemployment data
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv", dtype={"fips": str})

# Add a new category 'custom_category' to each feature in the GeoJSON data
for feature in counties['features']:
    fips = feature['properties']['GEO_ID'][9:]  # Extract FIPS code from GEO_ID
    if fips in df['fips'].values:
        unemp_value = df[df['fips'] == fips]['unemp'].values[0]  # Get 'unemp' value for the FIPS code
        feature['properties']['custom_category'] = unemp_value
    else:
        feature['properties']['custom_category'] = None  # or any default value if 'fips' not in df


In [ ]:
feature['properties']